In [ ]:
import pandas as pd
import joblib
import nltk
from nltk.tokenize import sent_tokenize
from bert_score import score

nltk.download("punkt")


In [ ]:
# Load data (must contain 'text' and 'summaries' columns)
df = pd.read_csv("cleaned_scientific_dataset.csv")
df = df.dropna(subset=["text", "summaries"]).reset_index(drop=True)
df_sample = df.sample(n=20, random_state=42)


In [ ]:
# Load trained Random Forest model and TF-IDF vectorizer
model = joblib.load("random_forest_summary_model.joblib")
vectorizer = joblib.load("tfidf_vectorizer.joblib")


In [ ]:
# Define extractive summarization function
def summarize_with_rf(article, model, vectorizer, top_k=None):
    sentences = sent_tokenize(article)
    if not sentences:
        return ""
    
    X = vectorizer.transform(sentences)
    preds = model.predict(X)
    
    selected = [s for s, p in zip(sentences, preds) if p == 1]
    if top_k:
        selected = selected[:top_k]
    
    return " ".join(selected)


In [ ]:
# Generate summaries and prepare for evaluation
generated = []
references = []

for _, row in df_sample.iterrows():
    ref = row['summaries']
    article = row['text']
    summary = summarize_with_rf(article, model, vectorizer)
    
    if summary.strip():
        generated.append(summary)
        references.append(ref)


In [ ]:
# Evaluate using BERTScore
P, R, F1 = score(generated, references, lang="en", verbose=True)

print("\n🧪 BERTScore Results:")
print(f"Precision: {P.mean():.4f}")
print(f"Recall:    {R.mean():.4f}")
print(f"F1 Score:  {F1.mean():.4f}")
